# Global GDP Trend

## What is GDP?
Gross domestic product (GDP) is the total monetary or market value of all the finished goods and services produced within a country's borders in a specific time period. As a broad measure of overall domestic production, it functions as a comprehensive scorecard of a given country's economic health.

Source of data: https://data.worldbank.org/indicator/NY.GDP.MKTP.KD.ZG

In [ ]:
#!pip install geopy
!pip install pycountry-convert

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import iplot

#from geopy.geocoders import Nominatim
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

In [ ]:
#Load data
df = pd.read_csv('/kaggle/input/gdp-annual-growth-for-each-country-1960-2020/GDP_annual_growth.csv')
gdp_data = df.drop(['Country Code','Indicator Name','Indicator Code'],axis=1)

## Trend of average GDP of the World

In [ ]:
# Global trend from 1960 to 2019
gdp_data = pd.melt(gdp_data,id_vars='Country Name',value_vars=gdp_data.columns[1:-2])
gdp_data.columns = ['Country','Year','GDP']
world_gdp = gdp_data[gdp_data['Country'] == 'World'].reset_index()[['Year','GDP']]

avg_gdp = world_gdp.GDP
growth_rate = []
for i in range(59):
    growth_rate.append(round(((avg_gdp[i+1]-avg_gdp[i])/avg_gdp[i])*100,3))
growth_rate.append(np.nan)
world_gdp['Growth_rate'] = growth_rate

In [ ]:
def create_fig(graph_obj,title='',x_title='',y_title=''):
    '''
    This function takes in a graph object and return a figure
    '''
    data = [graph_obj]
    layout = dict(title = dict(text=title),
              xaxis= dict(title=x_title,zeroline= False),
              yaxis= dict(title=y_title,zeroline= False)
             )
    fig = dict(data=data,layout=layout)
    return fig


def get_trend_plot(loc,country=False):
    df = gdp_data[gdp_data['Country'] == loc].reset_index()[['Year','GDP']]
    gdp_series = df.GDP
    growth_rate = []
    for i in range(59):
        growth_rate.append(round(((gdp_series[i+1]-gdp_series[i])/gdp_series[i])*100,3))
    growth_rate.append(np.nan)
    df['Growth_rate'] = growth_rate
    
    gdp_trace = go.Scatter(
    x = df.Year,
    y = df.GDP,
    mode = "lines+markers",
    marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
    text= 'in USD')
    
    growth_rate_trace = go.Scatter(
    x = df.Year,
    y = df.Growth_rate,
    mode = "lines+markers",
    marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
    text= 'GDP growth rate (%)')
    
    if(country):
        return gdp_trace,growth_rate_trace
    
    fig1_title = 'GDP Trend: ' + loc
    fig2_title = 'GDP growth rate Trend: ' + loc
    fig1 = create_fig(gdp_trace,fig1_title,'Year','GDP in USD')
    fig2 = create_fig(growth_rate_trace,fig2_title,'Year','GDP growth rate (%)')
    
    return fig1, fig2

<hr/>

**Below, we see the variation of average GDP  of the world and its rate of growth over the last 4 decades. It can be observed that even though there is an constant increment in the global average GDP with a couple of exception, there is an ever-existent randomness and unpredicatable nature in the growth rate.<br><br><br>
One of the dips in the global average is seen in the year of 2008 which saw The Great Recession (sometimes referred to as the 2008 Recession) in the United States and Western Europe which is linked to the so-called “subprime mortgage crisis.”
<br>**
![](https://www.outlookindia.com/outlookmoney/public/uploads/article/gallery/7f270a32de8158a1c2bc4ff7364f2cf6.jpg)
<br>
<br><br>
**The other significant dip is seen in 2014, which saw the Russian Financial Crisis, global oil prices took a nosedive in June 2014. The average price for a barrel of oil dropped nearly 40% in six months from the previous 100 USD threshold.**
<br>
![](https://cdn3.vox-cdn.com/uploads/chorus_asset/file/3332718/oilprice_jan23.0.png)
<br>

In [ ]:
gdpTrend,rateTrend = get_trend_plot('World')
iplot(gdpTrend)
iplot(rateTrend)

Country specific Trends

In [ ]:
def get_continent(col):
    try:
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return (cn_a2_code, cn_continent)

renaming_dict = {'Bahamas, The':'Bahamas',"Cote d'Ivoire":'Ivory Coast','Congo, Dem. Rep.':'Democratic Republic of the Congo',
                 'Congo, Rep.':'Republic of the Congo','Curacao':'Curaçao','Egypt, Arab Rep.':'Egypt','Gambia, The':'Gambia',
                 'Hong Kong SAR, China':'Hong Kong','Iran, Islamic Rep.':'Iran','Korea, Rep.':'South Korea','Lao PDR':'Laos',
                 'Macao SAR, China':'Macau','Korea, Dem. People’s Rep.':'North Korea','St. Vincent and the Grenadines':'Saint Vincent and the Grenadines',
                'Venezuela, RB':'Venezuela','Yemen, Rep.':'Yemen'}
for key in renaming_dict:
    gdp_data.loc[gdp_data['Country']==key,'Country'] = renaming_dict[key]
    
gdp_data['CountryCode'] = gdp_data['Country'].apply(get_continent)
gdp_data=gdp_data[gdp_data.CountryCode!=('Unknown','Unknown')]

country_code = gdp_data.apply(lambda x: x.CountryCode[0],axis=1)
cont_code = gdp_data.apply(lambda x: x.CountryCode[1],axis=1)
gdp_data['CountryCode'] = country_code
gdp_data['ContCode'] = cont_code

In [ ]:
from plotly.subplots import make_subplots
countries = sorted(set(gdp_data.Country))

fig=fig = make_subplots(rows=2, cols=1)

region_plot_names = []
buttons=[]

default_country = "United States"

for country_name in countries:
    country_df = gdp_data[gdp_data.Country==country_name].reset_index()
    avg_gdp = country_df.GDP
    growth_rate = []
    for i in range(59):
        growth_rate.append(round(((avg_gdp[i+1]-avg_gdp[i])/avg_gdp[i])*100,3))
    growth_rate.append(np.nan)
    country_df['Growth_rate'] = growth_rate
    fig.append_trace(go.Scatter(x = country_df.Year,
                             y = country_df.GDP,
                             mode = "lines+markers",
                             marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                             text= 'in USD', 
                             visible=(country_name==default_country)),row=1, col=1)
    fig.append_trace(go.Scatter(x = country_df.Year,
                             y = country_df.Growth_rate,
                             mode = "lines+markers",
                             marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                             text= 'in %', 
                             visible=(country_name==default_country)),row=2, col=1)
  
    region_plot_names.extend([country_name]*2)
    buttons.append(dict(method='update',
                        label=country_name,
                        args = [{'visible': [country_name==r for r in region_plot_names]}]))

fig.update_layout(showlegend=False, updatemenus=[{"buttons": buttons, "direction": "down", "active": countries.index(default_country), "showactive": True, "x": 0.5, "y": 1.15}])
fig.show()

Map with dropdown to select Year

In [ ]:
# geolocator = Nominatim(user_agent="kaggle")
# def geolocate(country):
#     try:
#         # Geolocate the center of the country
#         loc = geolocator.geocode(country)
#         # And return latitude and longitude
#         return (loc.latitude, loc.longitude)
#     except:
#         # Return missing value
#         return np.nan
    

# gdp_data['LatLong'] = gdp_data['CountryCode'].apply(geolocate)
# latitude = gdp_data.apply(lambda x: x.LatLong[0],axis=1)
# longitude = gdp_data.apply(lambda x: x.LatLong[1],axis=1)
# gdp_data['Lat'] = country_code
# gdp_data['Long'] = cont_code